Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the cells accordingly.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "曾致崴"
COLLABORATORS = "王靖淳"

---

# Introducing PyTorch

Parts of the materials are from the [Brains, Minds and Machines summer course 2018](http://cbmm.mit.edu/summer-school/2018).

Some materials are adpated from Dive into Deep Learning
https://d2l.ai/ 


PyTorch is an open source library for numerical computation using  computation graphs. Nodes in the graph represent mathematical operations, while the graph edges represent the multidimensional data arrays (tensors) communicated between them. 

Similar to python programming, we can add and execute a node to the computation graph immediately. This property makes it easy to debug the code and inspect the values in the network.

PyTorch provides several useful modules.

*  [ ```torch.nn```](https://pytorch.org/docs/stable/nn.html): This module provides the building blocks to create the networks, including the implementation of the various layers. 
*  [```torch.optim```](https://pytorch.org/docs/stable/optim.html): This module provides various optimization algorithms. Most commonly used methods are already supported.


In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

## Tensors

Tensors are the leaf variables in the computation graph. By default, a tensor is initialized randomly. 

You can perform the operations defined in [```torch.Tensor```](https://pytorch.org/docs/stable/tensors.html) on the tensors.

In [3]:
A = torch.Tensor(5, 3)
B = torch.rand(5, 3)
C = torch.Tensor(5, 3).uniform_(-1, 1)

print(A)
print(B)
print(C)
print(A + B)
print(torch.add(A, B))
print(C.size())

tensor([[-6.8878e-12,  3.0848e-41,  3.7835e-44],
        [ 0.0000e+00,         nan,  3.0848e-41],
        [ 1.3733e-14,  6.4069e+02,  4.3066e+21],
        [ 1.1824e+22,  4.3066e+21,  6.3828e+28],
        [ 3.8016e-39,  3.0848e-41, -1.4202e-12]])
tensor([[0.2150, 0.0450, 0.9069],
        [0.5880, 0.3780, 0.9628],
        [0.9493, 0.3219, 0.1990],
        [0.5501, 0.5545, 0.5037],
        [0.4084, 0.4961, 0.1181]])
tensor([[ 0.7136,  0.9217, -0.9703],
        [ 0.9994, -0.7058,  0.2503],
        [-0.8003, -0.1351, -0.2828],
        [-0.4698,  0.9856,  0.8299],
        [-0.4351,  0.6965,  0.2197]])
tensor([[2.1499e-01, 4.5021e-02, 9.0694e-01],
        [5.8801e-01,        nan, 9.6284e-01],
        [9.4933e-01, 6.4101e+02, 4.3066e+21],
        [1.1824e+22, 4.3066e+21, 6.3828e+28],
        [4.0835e-01, 4.9613e-01, 1.1808e-01]])
tensor([[2.1499e-01, 4.5021e-02, 9.0694e-01],
        [5.8801e-01,        nan, 9.6284e-01],
        [9.4933e-01, 6.4101e+02, 4.3066e+21],
        [1.1824e+22, 4.3066e

### Tensor Types
source: http://pytorch.org/docs/master/tensors.html

|Data type |CPU Tensor| 
|----------|------|
|32-bit floating point		|torch.FloatTensor	|
|64-bit floating point		|torch.DoubleTensor	|
|16-bit floating point		|torch.HalfTensor	|
|8-bit integer (unsigned)	|torch.ByteTensor	| 
|8-bit integer (signed)		|torch.CharTensor	|
|16-bit integer (signed)	|torch.ShortTensor	|
|32-bit integer (signed)	|torch.IntTensor	|
|64-bit integer (signed)	|torch.LongTensor	|

In [4]:
# Data type inferred from numpy
print(torch.from_numpy(np.random.rand(5, 3)).type())
print(torch.from_numpy(np.random.rand(5, 3).astype(np.float32)).type())

torch.DoubleTensor
torch.FloatTensor


### Reshape

In [5]:
y = torch.randn(5, 10, 15)
print(y.size())
print(y.view(-1, 15).size())  # Same as doing y.view(50, 15)
print(y.view(-1, 15).unsqueeze(1).size()) # Adds a dimension at index 1.
print(y.view(-1, 15).unsqueeze(1).squeeze().size())
# If input is of shape: (Ax1xBxCx1xD)(Ax1xBxCx1xD) then the out Tensor will be of shape: (AxBxCxD)(AxBxCxD)
print()
print(y.transpose(0, 1).size())
print(y.transpose(1, 2).size())
print(y.transpose(0, 1).transpose(1, 2).size())
print(y.permute(1, 2, 0).size())

torch.Size([5, 10, 15])
torch.Size([50, 15])
torch.Size([50, 1, 15])
torch.Size([50, 15])

torch.Size([10, 5, 15])
torch.Size([5, 15, 10])
torch.Size([10, 15, 5])
torch.Size([10, 15, 5])


### Repeat

In [6]:
print(y.view(-1, 15).unsqueeze(1).expand(50, 100, 15).size())
print(y.view(-1, 15).unsqueeze(1).expand_as(torch.randn(50, 100, 15)).size())

torch.Size([50, 100, 15])
torch.Size([50, 100, 15])


## Numpy Bridge

You can convert a tensor to a numpy array easily and vice versa. The torch Tensor and numpy array will share their underlying memory locations, and changing one will change the value of the other.

In [7]:
# convert pytorch tensor to numpy
a = torch.ones(5)
print(a.numpy())
a.numpy()[1] = 2
print(a)

# convert numpy to pytorch tensor
b = np.ones(5)
print(torch.from_numpy(b))

[1. 1. 1. 1. 1.]
tensor([1., 2., 1., 1., 1.])
tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


## Computing Gradient with Autograd

PyTorch uses a technique called automatic differentiation. That is, we have a recorder that records what operations we have performed, and then it replays it backward to compute our gradients. This technique is especially powerful when building neural networks, as we save time on one epoch by calculating differentiation of the parameters at the forward pass itself.

Once you finish your computation in the forward pass, you can call ```.backward()``` and have all the gradients computed automatically. You can access the gradient w.r.t. this tensor in ```.grad```.


Create a tensor and set ``requires_grad=True`` to track computation with it

In [8]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation:


In [9]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


``y`` was created as a result of an operation, so it has a ``grad_fn``.

In [10]:
print(y.grad_fn)

Do more operations on ``y``


In [11]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


Gradients
---------
Let's backprop now.
Because ``out`` contains a single scalar, ``out.backward()`` is
equivalent to ``out.backward(torch.tensor(1.))``.

In [12]:
# compute gradients
out.backward()
# print gradients d(out)/dx
print(x.grad.data)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

## Using GPU

To train on the GPU, we need to move all tensors and layers to the GPU first. You can do this by calling ```.cuda()``` or create a tensor with a device location. If you have multiple GPUs, you can select which GPU you want to use by setting ```cuda:n``` where n is the device id of the GPU. 

In [13]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)                       # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # ``.to`` can also change dtype together!

tensor([[2., 2.],
        [2., 2.]], device='cuda:0', grad_fn=<AddBackward0>)
tensor([[2., 2.],
        [2., 2.]], dtype=torch.float64, grad_fn=<ToCopyBackward0>)


### Part A: Move tensors on the CPU -> GPU (1 point)

In [22]:
x = torch.FloatTensor(5, 3).uniform_(-1, 1)
print(x)

# YOUR CODE HERE
device = torch.device("cuda")
a = torch.ones_like(x, device=device)
x = x.to(device)
# print(x)

raise NotImplementedError()

tensor([[ 0.1610,  0.3224,  0.4260],
        [ 0.3430,  0.8005, -0.1152],
        [ 0.1741,  0.7376, -0.4905],
        [ 0.3780, -0.5211, -0.8933],
        [-0.9146, -0.6578, -0.9438]])


NotImplementedError: ignored

In [20]:
assert x.get_device() == 0

## GD vs. SGD

### Gradient Descent

In this section we are going to introduce the basic concepts underlying gradient descent. An understanding of gradient descent is key to understanding stochastic gradient descent algorithms. For instance, the optimization problem might diverge due to an overly large learning rate. This phenomenon can already be seen in gradient descent. Likewise, preconditioning is a common technique in gradient descent and carries over to more advanced algorithms. Let us start with a simple special case. Gradient descent in one dimension is an excellent example to explain why the gradient descent algorithm may reduce the value of the objective function.

In [26]:
!pip install d2l

In [27]:
%matplotlib inline
from d2l import torch as d2l
import numpy as np
import torch

In [28]:
f = lambda x: x**2  # Objective function
gradf = lambda x: 2 * x  # Its derivative

Next, we use $x=10$ as the initial value and assume $\eta=0.2$. Using gradient descent to iterate $x$ for 10 times we can see that, eventually, the value of $x$ approaches the optimal solution.


In [29]:
def gd(eta):
    x = 10.0
    results = [x]
    for i in range(10):
        x -= eta * gradf(x)
        results.append(float(x))
    print('epoch 10, x:', x)
    return results

res = gd(0.2)

epoch 10, x: 0.06046617599999997


The progress of optimizing over $x$ can be plotted as follows.


In [31]:
def show_trace(res):
    n = max(abs(min(res)), abs(max(res)))
    f_line = torch.arange(-n, n, 0.01)
    d2l.set_figsize()
    d2l.plot([f_line, res], [[f(x) for x in f_line], [f(x) for x in res]], 'x', 'f(x)', fmts=['-', '-o'])

show_trace(res)

ImportError: ignored

<Figure size 252x180 with 1 Axes>

#### Learning Rate

The learning rate $\eta$ can be set by the algorithm designer. If we use a learning rate that is too small, it will cause $x$ to update very slowly, requiring more iterations to get a better solution. To show what happens in such a case, consider the progress in the same optimization problem for $\eta = 0.05$. As we can see, even after 10 steps we are still very far from the optimal solution.


In [ ]:
show_trace(gd(0.05))

Conversely, if we use an excessively high learning rate, $\left|\eta f'(x)\right|$ might be too large for the first-order Taylor expansion formula. That is, the term $\mathcal{O}(\eta^2 f'^2(x))$ in :eqref:`gd-taylor` might become significant. In this case, we cannot guarantee that the iteration of $x$ will be able to lower the value of $f(x)$. For example, when we set the learning rate to $\eta=1.1$, $x$ overshoots the optimal solution $x=0$ and gradually diverges.


In [ ]:
show_trace(gd(1.1))

#### Local Minima

To illustrate what happens for nonconvex functions consider the case of $f(x) = x \cdot \cos c x$. This function has infinitely many local minima. Depending on our choice of learning rate and depending on how well conditioned the problem is, we may end up with one of many solutions. The example below illustrates how an (unrealistically) high learning rate will lead to a poor local minimum.


### Stochastic Gradient Updates

In deep learning, the objective function is usually the average of the loss functions for each example in the training dataset. We assume that $f_i(\mathbf{x})$ is the loss function of the training dataset with $n$ examples, an index of $i$, and parameter vector of $\mathbf{x}$, then we have the objective function

$$f(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n f_i(\mathbf{x}).$$

The gradient of the objective function at $\mathbf{x}$ is computed as

$$\nabla f(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n \nabla f_i(\mathbf{x}).$$

If gradient descent is used, the computing cost for each independent variable iteration is $\mathcal{O}(n)$, which grows linearly with $n$. Therefore, when the model training dataset is large, the cost of gradient descent for each iteration will be very high.

Stochastic gradient descent (SGD) reduces computational cost at each iteration. At each iteration of stochastic gradient descent, we uniformly sample an index $i\in\{1,\ldots, n\}$ for data examples at random, and compute the gradient $\nabla f_i(\mathbf{x})$ to update $\mathbf{x}$:

$$\mathbf{x} \leftarrow \mathbf{x} - \eta \nabla f_i(\mathbf{x}).$$

Here, $\eta$ is the learning rate. We can see that the computing cost for each iteration drops from $\mathcal{O}(n)$ of the gradient descent to the constant $\mathcal{O}(1)$. We should mention that the stochastic gradient $\nabla f_i(\mathbf{x})$ is the unbiased estimate of gradient $\nabla f(\mathbf{x})$.

$$\mathbb{E}_i \nabla f_i(\mathbf{x}) = \frac{1}{n} \sum_{i = 1}^n \nabla f_i(\mathbf{x}) = \nabla f(\mathbf{x}).$$

This means that, on average, the stochastic gradient is a good estimate of the gradient.

Now, we will compare it to gradient descent by adding random noise with a mean of 0 and a variance of 1 to the gradient to simulate a SGD.


In [ ]:
f = lambda x1, x2: x1 ** 2 + 2 * x2 ** 2  # Objective
gradf = lambda x1, x2: (2 * x1, 4 * x2)  # Gradient

def sgd(x1, x2, s1, s2):
    global lr  # Learning rate scheduler
    (g1, g2) = gradf(x1, x2)
    # Simulate noisy gradient
    g1 += torch.normal(0.0, 1, (1,))
    g2 += torch.normal(0.0, 1, (1,))
    eta_t = eta * lr()  # Learning rate at time t
    return (x1 - eta_t * g1, x2 - eta_t * g2, 0, 0)  # Update variables

eta = 0.1
lr = (lambda: 1)  # Constant learning rate
d2l.show_trace_2d(f, d2l.train_2d(sgd, steps=50))

In [ ]:
c = torch.tensor(0.15 * np.pi)
f = lambda x: x * torch.cos(c * x)
gradf = lambda x: torch.cos(c * x) - c * x * torch.sin(c * x)
show_trace(gd(2))

# Building Model: Linear Regression

Now, we know how to do the basic computation using PyTorch. Let's start to build a simple linear regression model.

First, we generate 100 x, y data points for training. The data points follow this formula: y = 3.0 * x + 1.0

The goal is to regress W and b that fits the training data.

## Data Preparation

In [ ]:
x_train = np.random.rand(100).astype(np.float32).reshape(-1,1)
y_correct = 3.0 * x_train + 1.0

plt.plot(x_train, y_correct)

A model in PyTorch is a subclass of ```nn.Module```. There are several predefined layers and containers in the [ ```torch.nn```](https://pytorch.org/docs/stable/nn.html) module. We can use those to build more complex network structures.

You can check out the layers we talked about in the lecture here:

*   [```nn.Linear```](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear): fully connected layer
*   [```nn.Conv2d```](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d): convolution layer
* [```nn.MaxPool2d```](https://pytorch.org/docs/stable/nn.html#torch.nn.MaxPool2d): pooling
* [```nn.LSTM```](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM): LSTM unit

More complex neural networks are easily built using the predefined layers with the container classes ```Sequential``` and ```Concat```. 

The main method to implement in the ```nn.Module``` is ```forward```. It computes the output of the model given the input Tensor. The computation we put here defines the network architecture and how data flows between layers.

## Forward to compute prediction

A linear regression model is basically a single linear layer!

We first define the layers we needed in ```__init__``` and then build the computation in ```forward```.

In [ ]:
class LinearRegressionModel(nn.Module):

    def __init__(self, input_dim, output_dim):

        super(LinearRegressionModel, self).__init__() 
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

We then instantiate the model for training. We will use the mean squared error ([`nn.MSELoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss)) as loss function and optimize the network with stochastic gradient descent ([`torch.optim.SGD`](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD)).

We need to specify the parameters we want to optimize in the first argument of the selected optimizer. Usually, that should be all parameters in the model.

In [ ]:
model = LinearRegressionModel(1, 1)

criterion = nn.MSELoss()

## Backprop

### Train the Model

Similar to what we did in the numpy example, at each training iteration, we perform one forward pass, get the loss, compute the gradients (using ```.backward()```), and update the parameters by taking a ```.step()``` with the optimizer.

**Note** that we need to clear the gradients that are accumulated from previous training iterations first so we don't optimize against the old gradients stored in the variables. We can easily do it by calling ```optimizer.zero_grad()``` at the beginning of each iteration. 



[Tensorboard](https://www.tensorflow.org/guide/summaries_and_tensorboard) is a utility to record and visualize the activations, weights, loss, etc of your network, during and after training. It greatly simplifies debugging and iterative development of the network architecture. It is highly recommended.

To use TensorBoard in this notebook, we have to overcome two technical hurdles:
 1. we will be using [PyTorch](https://pytorch.org/) to work with our networks, but as the name suggests, TensorBoard is designed to work with [TensorFlow](https://www.tensorflow.org/), a competing framework.
 2. generally it is easiest to use TensorBoard when you're working on your local computer or on a computer on your local network, but in this tutorial we will run these notebooks on remote virtual machines via Colab.

We will use an extension in Colab.

In [ ]:
from tensorflow import summary

%load_ext tensorboard

Let's do the training again and write summary to tensorboard. Let's get summary writers for traininng and testing.

In [ ]:
import datetime

current_time = str(datetime.datetime.now().timestamp())
train_log_dir = 'logs/tensorboard/train/' + current_time
test_log_dir = 'logs/tensorboard/test/' + current_time
train_summary_writer = summary.create_file_writer(train_log_dir)
#test_summary_writer = summary.create_file_writer(test_log_dir)

### Part B: Please fill in the training code

* zero_grad(): zero the gradients (this is needed because PyTorch accumulates gradients in all parameters every time we call loss.backward())
* forward: forward propagation of inputs: compute the activation of all units within the network
* compute the loss 
* backpropagation: compute the numerical gradient of the loss with respect to all parameters, evaluated for the current values of the activations and the loss. As mentioned above, these values are stored locally within each parameter.
* step() to update weights: take a step with the optimizer we have defined above. The optimizer will go through all the parameters it know about (which we have specified when we created it), and update them according to the gradients it will find there, which we have computed when we performed backpropagation.

In [ ]:
model = LinearRegressionModel(1, 1)

criterion = nn.MSELoss()
learing_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learing_rate)

epochs = 2000

for epoch in range(epochs):
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_correct)

    # YOUR CODE HERE
    raise NotImplementedError()

    with train_summary_writer.as_default():
        summary.scalar('loss', loss.item(), epoch)

    if epoch % 100 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))


In [ ]:
%tensorboard --logdir logs/tensorboard

### After the Training

After the model is trained, we can apply the ```forward``` method to get the prediction of any data point.

We can also visualize it to see what the predictions look like!

In [ ]:
predicted = model.forward(torch.from_numpy(x_train)).data.numpy()

plt.plot(x_train, y_correct, 'go', label='from data', alpha=0.5)
plt.plot(x_train, predicted, label='prediction', alpha=0.5)
plt.legend()

In case anything goes wrong and you want to check the learned parameters of your model, you can inspect the `weight` and `bias` of a layer as follows. 

### Part B: inspect the weight and bias (1 point)

In [ ]:
print(model.linear.weight)
print(model.linear.bias)

assert abs(model.linear.weight.detach().numpy() -3) < 0.2 
assert abs(model.linear.bias.detach().numpy() -1) < 0.2 